In [2]:
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import librosa
from sklearn.model_selection import train_test_split

from transformers import AutoModelForAudioClassification, AutoFeatureExtractor

from src.const import AUDIO_PATH, MAIN_LABELS, BATCH_SIZE, VALIDATION_SPLIT, SEED
from src.preprocess import load_and_preprocess, transform_to_data_loader, get_dl_for_pretrained
from src.preprocess_utils import normalize_ds

c:\Users\izate\anaconda3\envs\dl2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\izate\anaconda3\envs\dl2\lib\site-packages\transformers\utils\generic.py:485: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


### Unknown vs known task

In [2]:
train_ds_bin_X, train_ds_bin_y, val_ds_bin_X, val_ds_bin_y, train_ds_main_X, train_ds_main_y, val_ds_main_X, val_ds_main_y = load_and_preprocess(plot_samples = False, augment_specs = False)

Loading augmented data...
Found 64727 files belonging to 31 classes.
Using 51782 files for training.
Using 12945 files for validation.
Finished
Creating data with only main classes...
Finished
Creating binary dataset...
Finished
Creating main dataset...
Found 64727 files belonging to 31 classes.
Using 51782 files for training.
Using 12945 files for validation.
Finished
Transforming audio data to spectograms...
Finished
Transforming data to numpy arrays...


Processing dataset: 100%|██████████| 12945/12945 [00:28<00:00, 454.01it/s]


Finished


In [3]:
train_ds_bin_X[train_ds_bin_X == -np.inf] = np.min(train_ds_bin_X[train_ds_bin_X != -np.inf])
val_ds_bin_X[val_ds_bin_X == -np.inf] = np.min(train_ds_bin_X[train_ds_bin_X != -np.inf])
train_ds_main_X[train_ds_main_X == -np.inf] = np.min(train_ds_main_X[train_ds_main_X != -np.inf])
val_ds_main_X[val_ds_main_X == -np.inf] = np.min(train_ds_main_X[train_ds_main_X != -np.inf])

In [4]:
train_ds_bin_X = normalize_ds(train_ds_bin_X)
val_ds_bin_X = normalize_ds(val_ds_bin_X)
train_ds_main_X = normalize_ds(train_ds_main_X)
val_ds_main_X = normalize_ds(val_ds_main_X)

Normalizing dataset:   0%|          | 0/70776 [00:00<?, ?it/s]

Normalizing dataset: 100%|██████████| 4688/4688 [00:00<00:00, 7417.25it/s]


In [5]:
# np.save('data/arrays/train_ds_bin_X.npy', train_ds_bin_X)
# np.save('data/arrays/train_ds_bin_y.npy', train_ds_bin_y)
# np.save('data/arrays/val_ds_bin_X.npy', val_ds_bin_X)
# np.save('data/arrays/val_ds_bin_y.npy', val_ds_bin_y)
# np.save('data/arrays/train_ds_main_X.npy', train_ds_main_X)
# np.save('data/arrays/train_ds_main_y.npy', train_ds_main_y)
# np.save('data/arrays/val_ds_main_X.npy', val_ds_main_X)
# np.save('data/arrays/val_ds_main_y.npy', val_ds_main_y)

In [3]:
train_ds_bin_X = np.load('data/arrays/train_ds_bin_X.npy')
train_ds_bin_y = np.load('data/arrays/train_ds_bin_y.npy')
val_ds_bin_X = np.load('data/arrays/val_ds_bin_X.npy')
val_ds_bin_y = np.load('data/arrays/val_ds_bin_y.npy')
train_ds_main_X = np.load('data/arrays/train_ds_main_X.npy')
train_ds_main_y = np.load('data/arrays/train_ds_main_y.npy')
val_ds_main_X = np.load('data/arrays/val_ds_main_X.npy')
val_ds_main_y = np.load('data/arrays/val_ds_main_y.npy')

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
train_loader_bin = transform_to_data_loader(train_ds_bin_X, train_ds_bin_y, device=device)
val_loader_bin = transform_to_data_loader(val_ds_bin_X, val_ds_bin_y, device=device)

train_loader_main = transform_to_data_loader(train_ds_main_X, train_ds_main_y, device=device)
val_loader_main = transform_to_data_loader(val_ds_main_X, val_ds_main_y, device=device)

del train_ds_bin_X, train_ds_bin_y, val_ds_bin_X, val_ds_bin_y, train_ds_main_X, train_ds_main_y, val_ds_main_X, val_ds_main_y

In [6]:
for a, b in train_loader_bin:
    print(a.shape, b.shape)
    break


torch.Size([256, 63, 128]) torch.Size([256])


In [106]:
import torch
from torch import nn
import torch.nn.functional as f
from torch.autograd import Variable

KERNEL_SIZE = 3
PADDING = KERNEL_SIZE // 2

class ConvLSTMCell(nn.Module):
    """
    Generate a convolutional LSTM cell
    """

    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.Gates = nn.Conv2d(input_size + hidden_size, 4 * hidden_size, KERNEL_SIZE, padding=PADDING)

    def forward(self, input_, prev_state):

        # get batch and spatial sizes
        batch_size = input_.data.size()[0]
        spatial_size = input_.data.size()[2:]

        # generate empty prev_state, if None is provided
        if prev_state is None:
            state_size = [batch_size, self.hidden_size] + list(spatial_size)
            prev_state = (
                Variable(torch.zeros(state_size)).to(device),
                Variable(torch.zeros(state_size)).to(device)
            )

        prev_hidden, prev_cell = prev_state

        # data size is [batch, channel, height, width]
        stacked_inputs = torch.cat((input_, prev_hidden), 1)
        gates = self.Gates(stacked_inputs)

        # chunk across channel dimension
        in_gate, remember_gate, out_gate, cell_gate = gates.chunk(4, 1)

        # apply sigmoid non linearity
        in_gate = f.sigmoid(in_gate)
        remember_gate = f.sigmoid(remember_gate)
        out_gate = f.sigmoid(out_gate)

        # apply tanh non linearity
        cell_gate = f.tanh(cell_gate)

        # compute current cell and hidden state
        cell = (remember_gate * prev_cell) + (in_gate * cell_gate)
        hidden = out_gate * f.tanh(cell)

        return hidden, cell

In [123]:
class bin_BiLSTM(nn.Module):
    
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_size//2, 
            hidden_size=hidden_size//2, 
            batch_first=True, 
            bidirectional=True, 
            num_layers=1,
            dropout=0.5,
        )
        self.convlstm = ConvLSTMCell(input_size=16, hidden_size=1)
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(hidden_size, 128)  # *2 because of bidirectional
        self.fc2 = nn.Linear(128, 1)
        self.bc1 = nn.BatchNorm1d(hidden_size)
        self.bc2 = nn.BatchNorm1d(128)
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
        self.conv = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)

    def forward(self, x):
        # out = self.conv(x.unsqueeze(1))#.squeeze(1)
        # out = self.pool(out)
        # out = self.dropout(out)
        # out = self.relu(out)
        # out = torch.mean(out, dim=1)
        # out, _ = self.lstm(out)

        out, _ = self.convlstm(x, None)
        
        # # out = torch.sum(out, dim=1)
        # out = self.dropout(out[:, -1, :])
        # out = self.bc1(out)
        # out = self.fc1(out)
        # out = self.dropout(out)
        # out = self.relu(out)
        # out = self.bc2(out)
        # out = self.fc2(out)
        # out = self.sigmoid(out)
        return out

In [116]:
class main_BiLSTM(nn.Module):

    def __init__(self, input_size, hidden_size, num_class):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_size, 
            hidden_size=hidden_size, 
            batch_first=True, 
            bidirectional=True, 
            num_layers=2,
            dropout=0.3,
        )
        self.dropout = nn.Dropout(0.3)
        self.fc1 = nn.Linear(2*hidden_size, 128)  # *2 because of bidirectional
        self.fc2 = nn.Linear(128, num_class)
        self.bc1 = nn.BatchNorm1d(2*hidden_size)
        self.bc2 = nn.BatchNorm1d(128)
        self.relu = nn.ReLU()
        self.conv = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, padding=1)

    def forward(self, x):
        out = self.conv(x.unsqueeze(1)).squeeze(1)
        out = self.dropout(out)
        out = self.relu(out)
        out = torch.min(out, dim=1)
        out, _ = self.lstm(out)
        # out = torch.sum(out, dim=1)
        out = self.bc1(out[:, -1, :])
        out = self.fc1(out)
        out = self.dropout(out)
        out = self.relu(out)
        out = self.bc2(out)
        out = self.fc2(out)
        out = self.dropout(out)
        return out

In [117]:
class bin_Transformer(nn.Module):
    
    def __init__(self, d_model, n_head, num_layers):
        super().__init__()
        self.trans_enc_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=n_head, batch_first=True, activation="relu").to(device)
        self.transformer_encoder = nn.TransformerEncoder(self.trans_enc_layer, num_layers=num_layers).to(device)
        self.dropout = nn.Dropout(0.2)
        self.fc1 = nn.Linear(d_model, 64)
        self.fc2 = nn.Linear(64, 1)
        self.bc1 = nn.BatchNorm1d(d_model)
        self.bc2 = nn.BatchNorm1d(64)
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
        self.conv = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, padding=1)

    def forward(self, x):
        out = self.conv(x.unsqueeze(1)).squeeze(1)
        out = self.dropout(out)
        out = self.relu(out)
        out = self.transformer_encoder(out)[:, -1, :]
        out = self.dropout(out)
        out = self.bc1(out)
        out = self.fc1(out)
        out = self.dropout(out)
        out = self.relu(out)
        out = self.bc2(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

In [118]:
class main_Transformer(nn.Module):
    
    def __init__(self, d_model, n_head, num_layers, num_class):
        super().__init__()
        self.trans_enc_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=n_head, batch_first=True, activation="relu").to(device)
        self.transformer_encoder = nn.TransformerEncoder(self.trans_enc_layer, num_layers=num_layers).to(device)
        self.dropout = nn.Dropout(0.1)
        self.fc1 = nn.Linear(d_model, 64)
        self.fc2 = nn.Linear(64, num_class)
        self.bc1 = nn.BatchNorm1d(d_model)
        self.bc2 = nn.BatchNorm1d(64)
        self.relu = nn.ReLU()
        self.conv = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)

    def forward(self, x):
        out = self.conv(x.unsqueeze(1)).squeeze(1)
        out = self.pool(out)
        out = self.dropout(out)
        out = self.relu(out)
        out = self.transformer_encoder(out)[:, -1, :]
        out = self.dropout(out)
        out = self.bc1(out)
        out = self.fc1(out)
        out = self.dropout(out)
        out = self.relu(out)
        out = self.bc2(out)
        out = self.fc2(out)
        out = self.dropout(out)
        return out

In [124]:
# BILSTM for known vs. unknown classification task
input_size = 128
hidden_size = 128

bin_bilstm_model = bin_BiLSTM(input_size, hidden_size).to(device)
bin_bilstm_criterion = nn.BCELoss()
bin_bilstm_optimizer = optim.Adam(bin_bilstm_model.parameters(), weight_decay=0.005)

In [125]:
# BILSTM for main classification task
input_size = 128
hidden_size = 128
num_class = 10

main_bilstm_model = main_BiLSTM(input_size, hidden_size, num_class).to(device)
main_bilstm_criterion = nn.CrossEntropyLoss()
main_bilstm_optimizer = optim.Adam(main_bilstm_model.parameters())

In [126]:
from torchinfo import summary
summary(bin_bilstm_model, (16, 63, 128,), device="cuda")
# print(bin_bilstm_model)

RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: []

In [13]:
# Transformer for known vs. unknown task
d_model = 128
n_head = 8
num_layers = 1

bin_transformer_model = bin_Transformer(d_model, n_head, num_layers).to(device)
bin_transformer_criterion = nn.BCELoss()
bin_transformer_optimizer = optim.Adam(bin_transformer_model.parameters())

In [14]:
# Transformer for main task
d_model = 128
n_head = 8
num_class = 10
num_layers = 2

main_transformer_model = main_Transformer(d_model, n_head, num_layers, num_class).to(device)
main_transformer_criterion = nn.CrossEntropyLoss()
main_transformer_optimizer = optim.Adam(main_transformer_model.parameters())

In [33]:
def train_model(
        model, 
        criterion, 
        optimizer, 
        train_loader, 
        val_loader, 
        model_type, 
        epoch_count,
        using_pretrained=False,
        early_stopping=False,
    ):
    
    train_losses = []
    val_losses = []
    best_val_loss = float('inf')
    patience = 5
    no_improve_count = 0

    # Training
    for epoch in range(epoch_count):
        model.train()
        train_loss = 0
        correct_train = 0
        total_train = 0

        for batch_X, batch_y in tqdm(train_loader, f"Epoch {epoch+1}"):
            model.zero_grad()
            outputs = model(batch_X)

            if using_pretrained:
                outputs = outputs.logits

            if model_type == "bin":
                loss = criterion(outputs, batch_y.unsqueeze(1))
            else:
                loss = criterion(outputs, batch_y.long())

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            # Loss
            train_loss += loss.item() * batch_X.size(0)

            # Accuracy
            if model_type == "bin":
                predicted = (outputs > 0.5).float()
                correct_train += (predicted == batch_y.unsqueeze(1)).sum().item()
            else:
                predicted = torch.argmax(outputs, dim=1)
                correct_train += (predicted == batch_y).sum().item()
            
            
            total_train += batch_y.size(0)

        train_loss /= len(train_loader.dataset)
        train_acc = correct_train / total_train
        train_losses.append(train_loss)
        
        model.eval()
        val_loss = 0
        correct_val = 0
        total_val = 0

        with torch.no_grad():
            for batch_X, batch_y in val_loader:
                outputs = model(batch_X)

                if using_pretrained:
                    outputs = outputs.logits

                if model_type == "bin":
                    loss = criterion(outputs, batch_y.unsqueeze(1))
                else:
                    loss = criterion(outputs, batch_y.long())
                
                # Loss
                val_loss += loss.item() * batch_X.size(0)

                # Accuracy
                if model_type == "bin":
                    predicted = (outputs > 0.5).float()
                    correct_val += (predicted == batch_y.unsqueeze(1)).sum().item()
                else:
                    predicted = torch.argmax(outputs, dim=1)
                    correct_val += (predicted == batch_y).sum().item()
                total_val += batch_y.size(0)

            val_loss /= len(val_loader.dataset)
            val_acc = correct_val / total_val
            val_losses.append(val_loss)
            
        print(f'Epoch {epoch+1}/{epoch_count}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}')
        
        
        if not early_stopping:
            continue
        
        # Early stopping check
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            no_improve_count = 0
        else:
            no_improve_count += 1
        
        if no_improve_count >= patience:
            print('Early stopping')
            break
            
    return train_losses, val_losses

### Custom architectures

In [16]:
# Known vs. Unknown with transformer
train_losses, val_losses = train_model(
    model=bin_transformer_model,
    criterion=bin_transformer_criterion,
    optimizer=bin_transformer_optimizer,
    train_loader=train_loader_bin,
    val_loader=val_loader_bin, 
    model_type="bin", 
    epoch_count=50
)

Epoch 1:   0%|          | 0/277 [00:00<?, ?it/s]c:\Users\izate\anaconda3\envs\dl2\lib\site-packages\torch\nn\functional.py:5476: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)
Epoch 1: 100%|██████████| 277/277 [00:08<00:00, 31.16it/s]


Epoch 1/50, Train Loss: 0.6974, Train Acc: 0.52, Val Loss: 0.6816, Val Acc: 0.60


Epoch 2: 100%|██████████| 277/277 [00:07<00:00, 35.10it/s]


Epoch 2/50, Train Loss: 0.6675, Train Acc: 0.58, Val Loss: 0.6814, Val Acc: 0.61


Epoch 3: 100%|██████████| 277/277 [00:07<00:00, 34.74it/s]


...

Epoch 48/50, Train Loss: 0.5819, Train Acc: 0.67, Val Loss: 0.6925, Val Acc: 0.61


Epoch 49: 100%|██████████| 277/277 [00:08<00:00, 34.37it/s]


Epoch 49/50, Train Loss: 0.5769, Train Acc: 0.67, Val Loss: 0.6878, Val Acc: 0.61


Epoch 50: 100%|██████████| 277/277 [00:07<00:00, 34.70it/s]


Epoch 50/50, Train Loss: 0.5766, Train Acc: 0.67, Val Loss: 0.6723, Val Acc: 0.62


In [ ]:
# Main task with transformer
train_losses, val_losses = train_model(
    model=main_transformer_model,
    criterion=main_transformer_criterion,
    optimizer=main_transformer_optimizer,
    train_loader=train_loader_main,
    val_loader=val_loader_main, 
    model_type="main", 
    epoch_count=50
)

In [67]:
# Known vs. Unknown with BiLSTM
train_losses, val_losses = train_model(
    model=bin_bilstm_model,
    criterion=bin_bilstm_criterion,
    optimizer=bin_bilstm_optimizer,
    train_loader=train_loader_bin,
    val_loader=val_loader_bin, 
    model_type="bin", 
    epoch_count=50
)

Epoch 1: 100%|██████████| 277/277 [00:16<00:00, 16.41it/s]


Epoch 1/50, Train Loss: 0.6960, Train Acc: 0.52, Val Loss: 0.7122, Val Acc: 0.38


Epoch 2: 100%|██████████| 277/277 [00:12<00:00, 22.29it/s]


Epoch 2/50, Train Loss: 0.6859, Train Acc: 0.53, Val Loss: 0.7117, Val Acc: 0.39


Epoch 3: 100%|██████████| 277/277 [00:09<00:00, 27.76it/s]


...

Epoch 27/50, Train Loss: 0.6286, Train Acc: 0.62, Val Loss: 0.7011, Val Acc: 0.60


Epoch 28: 100%|██████████| 277/277 [00:12<00:00, 21.44it/s]


Epoch 28/50, Train Loss: 0.6276, Train Acc: 0.62, Val Loss: 0.6961, Val Acc: 0.58


Epoch 29:  17%|█▋        | 47/277 [00:02<00:11, 20.84it/s]


KeyboardInterrupt: 

In [ ]:
# Main task with BiLSTM
train_losses, val_losses = train_model(
    model=main_bilstm_model,
    criterion=main_bilstm_criterion,
    optimizer=main_bilstm_optimizer,
    train_loader=train_loader_main,
    val_loader=val_loader_main, 
    model_type="main", 
    epoch_count=50
)

### Pretrained model

In [64]:
from transformers import AutoProcessor, WavLMModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

c:\Users\izate\anaconda3\envs\dl2\lib\site-packages\transformers\utils\generic.py:342: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [3]:
X, y = np.load("data.npy"), np.load("labels.npy")

In [5]:
X = X[y != 10]
y = y[y != 10]

In [6]:
X.shape

(18994, 16000)

In [7]:
feature_extractor = ASTFeatureExtractor()
X = np.array(feature_extractor(X, sampling_rate=16000)["input_values"])

In [8]:
X.shape

(18994, 1024, 128)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

In [15]:
train_dl = transform_to_data_loader(X_train, y_train, device=device)

In [16]:
val_dl = transform_to_data_loader(X_test, y_test, device=device)

In [3]:
# feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")
# feature_extractor = ASTFeatureExtractor()

# train_dl, val_dl = get_dl_for_pretrained(
#     feature_extractor=feature_extractor,
#     X_data_path="data.npy", 
#     y_data_path="labels.npy", 
#     device=device,
#     task_type="main"
# )

Extracting features for data with size (18994, 16000).
Splitting data with size (18994, 1024, 128).


In [25]:
# model = AutoModelForAudioClassification.from_pretrained("facebook/wav2vec2-base", num_labels=10).to(device)

configuration = ASTConfig()
model = ASTModel(configuration).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [27]:
train_losses, val_losses = train_model(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    train_loader=train_dl,
    val_loader=val_dl, 
    model_type="main", 
    epoch_count=50,
    using_pretrained=True
)

Epoch 1:   0%|          | 0/56 [00:02<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 910.00 MiB. GPU 0 has a total capacity of 6.00 GiB of which 0 bytes is free. Of the allocated memory 12.27 GiB is allocated by PyTorch, and 497.39 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)